<a href="https://colab.research.google.com/github/brinsga/Natural-Language-Processing/blob/master/Domain_Adaptation_combined_Siamese_lstm_question_pairs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Implememntation based on: https://www.kaggle.com/lystdo/lstm-with-word2vec-embeddings
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [12]:
!unzip drive/My\ Drive/NLP/quora_dataset/test.csv.zip 
!unzip drive/My\ Drive/NLP/quora_dataset/train.csv.zip 

Archive:  drive/My Drive/NLP/quora_dataset/test.csv.zip
  inflating: test.csv                
Archive:  drive/My Drive/NLP/quora_dataset/train.csv.zip
  inflating: train.csv               


In [0]:
BASE_DIR = 'drive/My Drive/NLP/quora_dataset/'

In [3]:
import os
import csv
import codecs
import numpy as np
import pandas as pd
np.random.seed(1337)

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten, merge, LSTM, Lambda, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers.wrappers import TimeDistributed, Bidirectional
from keras.layers.normalization import BatchNormalization
from keras import backend as K
import sys

Using TensorFlow backend.


In [0]:
!ls

drive		   glove.6B.200d.txt  glove.6B.50d.txt	sample_data  train.csv
glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip	test.csv


In [0]:
TRAIN_DATA_FILE = 'train.csv'
TEST_DATA_FILE = 'test.csv'
MAX_SEQUENCE_LENGTH = 30
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1

In [5]:
print('Indexing word vectors.')
embeddings_index = {}
f = codecs.open(os.path.join(BASE_DIR, 'glove.6B.300d.txt'), encoding='utf-8')
for line in f:
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [0]:
questions = {}
f = codecs.open(os.path.join('drive/My Drive/NLP/quora_dataset/apple/', 'corpus.tsv'), encoding='utf-8')
for line in f:
  values = line.strip().split("\t")
  questions[values[0]] = values[1]

f.close()

q1,q2,label = [],[],[]
f = codecs.open(os.path.join('drive/My Drive/NLP/quora_dataset/apple/', 'full.pos.txt'), encoding='utf-8')
for line in f:
  values = line.strip().split(" ")
  q1.append(questions[values[0]])
  q2.append(questions[values[1]])
  label.append(1)
f.close()

f = codecs.open(os.path.join('drive/My Drive/NLP/quora_dataset/apple/', 'full.neg.txt'), encoding='utf-8')
i = 0
for line in f:
  if i > 50000:
    break
  values = line.strip().split(" ")
  q1.append(questions[values[0]])
  q2.append(questions[values[1]])
  label.append(0)
  i+=1
f.close()

data = np.array(list(zip(q1,q2,label)))


In [0]:
np.random.seed(1)
np.random.shuffle(data)

In [0]:
train = data[5000:15000]
test = data[:5000]


In [0]:
q1_train = list(train[:,0])
q2_train = list(train[:,1])
label_train = list(train[:,2])

q1_test = list(test[:,0])
q2_test = list(test[:,1])
label_test = list(test[:,2])

In [13]:
print('Processing text dataset')
texts_1 = [] 
texts_2 = []
labels = []  # list of label ids
with codecs.open(TRAIN_DATA_FILE, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    header = next(reader)
    for values in reader:
        texts_1.append(values[3])
        texts_2.append(values[4])
        labels.append(int(values[5]))
print('Found %s texts.' % len(texts_1))

test_texts_1 = q1_test
test_texts_2 = q2_test
test_labels = label_test  # list of label ids

print('Found %s texts.' % len(test_texts_1))

Processing text dataset
Found 404290 texts.
Found 5000 texts.


In [21]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts_1 + texts_2)
sequences_1 = tokenizer.texts_to_sequences(texts_1)
sequences_2 = tokenizer.texts_to_sequences(texts_2)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

test_sequences_1 = tokenizer.texts_to_sequences(test_texts_1)
test_sequences_2 = tokenizer.texts_to_sequences(test_texts_2)

data_1 = pad_sequences(sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
data_2 = pad_sequences(sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
labels = np.array(labels)
print('Shape of data tensor:', data_1.shape)
print('Shape of label tensor:', labels.shape)

test_data_1 = pad_sequences(test_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
test_data_2 = pad_sequences(test_sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
test_labels = np.array(test_labels)
del test_sequences_1
del test_sequences_2
del sequences_1
del sequences_2
import gc
gc.collect()

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 95596 unique tokens.
Shape of data tensor: (404290, 30)
Shape of label tensor: (404290,)


0

In [22]:
print('Preparing embedding matrix.')
# prepare embedding matrix
nb_words = min(MAX_NB_WORDS, len(word_index))+1

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing embedding matrix.
Null word embeddings: 35260


In [0]:
embedding_layer = Embedding(nb_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [0]:
num_lstm = np.random.randint(175, 275)
num_dense = np.random.randint(100, 150)
rate_drop_lstm = 0.15 + np.random.rand() * 0.25
rate_drop_dense = 0.15 + np.random.rand() * 0.25

act = 'relu'
re_weight = True # whether to re-weight classes to fit the 17.5% share in test set

In [0]:
perm = np.random.permutation(len(data_1))
idx_train = perm[:int(len(data_1)*(1-VALIDATION_SPLIT))]
idx_val = perm[int(len(data_1)*(1-VALIDATION_SPLIT)):]

data_1_train = data_1[idx_train]
data_2_train = data_2[idx_train]
labels_train = labels[idx_train]

data_1_val = data_1[idx_val]
data_2_val = data_2[idx_val]
labels_val = labels[idx_val]

weight_val = np.ones(len(labels_val))
if re_weight:
    weight_val *= 0.472001959
    weight_val[labels_val==0] = 1.309028344

In [0]:
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, RNN, GRU
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [0]:
lstm_layer = LSTM(num_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)

sequence_1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer(embedded_sequences_1)

sequence_2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_2_input)
y1 = lstm_layer(embedded_sequences_2)

merged = concatenate([x1, y1])
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

merged = Dense(num_dense, activation=act)(merged)
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

preds = Dense(1, activation='sigmoid')(merged)

In [0]:
if re_weight:
    class_weight = {0: 1.309028344, 1: 0.472001959}
else:
    class_weight = None

In [0]:
model = Model(inputs=[sequence_1_input, sequence_2_input], \
        outputs=preds)
model.compile(loss='binary_crossentropy',
        optimizer='nadam',
        metrics=['acc',get_f1])

In [0]:
STAMP = 'lstm_%d_%d_%.2f_%.2f'%(num_lstm, num_dense, rate_drop_lstm, \
        rate_drop_dense)

In [38]:
#model.summary()
print(STAMP)

early_stopping =EarlyStopping(monitor='val_loss', patience=3)
bst_model_path = STAMP + '.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

hist = model.fit([data_1_train, data_2_train], labels_train, \
        validation_data=([data_1_val, data_2_val], labels_val, weight_val), \
        epochs=200, batch_size=2048, shuffle=True, \
        class_weight=class_weight, callbacks=[early_stopping, model_checkpoint])

model.load_weights(bst_model_path)
bst_val_score = min(hist.history['val_loss'])

lstm_225_125_0.33_0.39
Train on 363861 samples, validate on 40429 samples
Epoch 1/200
363861/363861 [==============================] - 40s 109us/step - loss: 0.4754 - acc: 0.6594 - get_f1: 0.3353 - val_loss: 0.3960 - val_acc: 0.6552 - val_get_f1: 0.1343
Epoch 2/200
363861/363861 [==============================] - 39s 108us/step - loss: 0.3808 - acc: 0.6894 - get_f1: 0.3189 - val_loss: 0.3617 - val_acc: 0.6893 - val_get_f1: 0.3003
Epoch 3/200
363861/363861 [==============================] - 39s 107us/step - loss: 0.3628 - acc: 0.7050 - get_f1: 0.3794 - val_loss: 0.3431 - val_acc: 0.7250 - val_get_f1: 0.4476
Epoch 4/200
363861/363861 [==============================] - 38s 104us/step - loss: 0.3510 - acc: 0.7159 - get_f1: 0.4183 - val_loss: 0.3341 - val_acc: 0.7364 - val_get_f1: 0.4874
Epoch 5/200
363861/363861 [==============================] - 38s 103us/step - loss: 0.3413 - acc: 0.7236 - get_f1: 0.4428 - val_loss: 0.3260 - val_acc: 0.7490 - val_get_f1: 0.5321
Epoch 6/200
363861/363861 

In [39]:
preds = model.predict([test_data_1, test_data_2], batch_size=500, verbose=1)

5000/5000 [==============================] - 0s 80us/step


In [0]:
new_labels = np.array(test_labels).astype(int).tolist()

In [67]:
new_preds = list(preds)
to_check = []
print(new_preds[1])
for i in range(len(new_preds)):
  if new_preds[i][0] > 0.1:
    to_check.append(1)
  else:
    to_check.append(0)


[0.03323421]


In [0]:
new_labels = list(new_labels)
acc = 0
for i in range(len(to_check)):
  if to_check[i] == new_labels[i]:
    acc +=1
acc = acc/ len(to_check)

In [69]:
#acc = 0.769
acc

0.8838

In [70]:
# 0.73346 #1000 training data
from sklearn.metrics import classification_report

y_test = list(np.array(new_labels).astype(int))
#y_pred_bool = np.argmax(new_preds, axis=1)
y_pred_bool = to_check

print(classification_report(y_test, y_pred_bool))

              precision    recall  f1-score   support

           0       0.97      0.91      0.94      4794
           1       0.11      0.26      0.15       206

    accuracy                           0.88      5000
   macro avg       0.54      0.58      0.55      5000
weighted avg       0.93      0.88      0.91      5000



In [71]:
#0.55

from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred_bool)

0.584001644450385